# Trendyol API Data Analysis and Processing

This notebook demonstrates how to process Trendyol API response data, with a focus on handling missing phone numbers and normalizing order data for our internal system.

In [1]:
import json
import pandas as pd
import re
from datetime import datetime

## Loading Sample Trendyol API Response

First, let's load a sample API response from Trendyol to analyze its structure

In [3]:
# Sample response data from Trendyol API
# This would typically be loaded from a file or API call
# For demonstration purposes, we'll use a simplified version

sample_response = {
    "totalElements": 48,
    "totalPages": 1,
    "page": 0,
    "size": 50,
    "content": []
}

# Load actual data from a JSON file
# In production, you would replace this with your actual API call
try:
    with open('trendyol_sample_response.json', 'r', encoding='utf-8') as f:
        sample_response = json.load(f)
    print(f"Loaded sample data with {len(sample_response.get('content', []))} orders")
except FileNotFoundError:
    print("Sample file not found. Using empty sample data.")
    # Create a minimal sample for demonstration
    sample_response['content'] = [{
        "shipmentAddress": {
            "firstName": "Sample",
            "lastName": "User",
            "address1": "Sample Address",
            "city": "Istanbul",
            "district": "Kadikoy",
            "phone": None,  # Note the null phone number
            "fullAddress": "Sample Address Line 1 Tel: 05551112233 Kadikoy Istanbul"
        },
        "orderNumber": "10219480975",
        "totalPrice": 349.00,
        "customerEmail": "sample@example.com",
        "customerFirstName": "Sample",
        "customerLastName": "User",
        "lines": [{
            "quantity": 1,
            "merchantSku": "SAMPLE-SKU",
            "productName": "Sample Product",
            "productCode": 1234567890,
            "barcode": "1234567890123",
            "price": 349.00
        }],
        "status": "Created",
        "orderDate": 1747599284403,
        "cargoProviderName": "Sample Cargo"
    }]

Sample file not found. Using empty sample data.


## Analyzing Phone Number Fields

The Trendyol API often returns `null` for the phone field in addresses. Let's analyze this pattern and create a robust extraction function.

In [4]:
def extract_phone_number(order):
    """
    Extract phone number from order data
    Handles various formats and null cases in Trendyol API responses
    
    Args:
        order (dict): The order object from Trendyol API
        
    Returns:
        str: The extracted phone number or empty string if not available
    """
    # Check if shipmentAddress phone field is available
    if order.get('shipmentAddress') and order['shipmentAddress'].get('phone'):
        return order['shipmentAddress']['phone']
    
    # Try to extract from fullAddress which sometimes contains phone
    full_address = order.get('shipmentAddress', {}).get('fullAddress', '')
    
    # Common patterns in Turkish addresses for phone numbers
    phone_patterns = [
        r'(?:tel|telefon|gsm|cep|phone)[:\s]*([+0-9\s()\-]{10,15})',  # Explicit markers
        r'\b(05\d{2}[\s-]?\d{3}[\s-]?\d{2}[\s-]?\d{2})\b',  # Turkish mobile format
        r'\b(\+?90[\s-]?5\d{2}[\s-]?\d{3}[\s-]?\d{2}[\s-]?\d{2})\b',  # With country code
        r'\b(\d{3}[\s-]?\d{3}[\s-]?\d{2}[\s-]?\d{2})\b'  # Generic number pattern
    ]
    
    for pattern in phone_patterns:
        match = re.search(pattern, full_address, re.IGNORECASE)
        if match:
            return match.group(1).strip()
    
    # Try invoiceAddress as fallback
    if order.get('invoiceAddress') and order['invoiceAddress'].get('phone'):
        return order['invoiceAddress']['phone']
    
    # If no phone found, return empty string
    return ''

# Test on the first order
if sample_response['content']:
    first_order = sample_response['content'][0]
    phone = extract_phone_number(first_order)
    print(f"Extracted phone: '{phone}'")
    print(f"Phone field in API: {first_order.get('shipmentAddress', {}).get('phone')}")

Extracted phone: '05551112233'
Phone field in API: None


## Order Status Mapping

Now let's create a function to map Trendyol order statuses to our internal system's statuses.

In [5]:
def map_order_status(trendyol_status):
    """
    Maps Trendyol order statuses to our internal system statuses
    
    Args:
        trendyol_status (str): The status from Trendyol API
        
    Returns:
        str: Mapped internal status
    """
    status_mapping = {
        'Created': 'NEW',
        'Awaiting': 'PENDING',
        'Picking': 'PROCESSING',
        'Shipped': 'SHIPPED',
        'Delivered': 'DELIVERED',
        'UnDelivered': 'FAILED_DELIVERY',
        'Cancelled': 'CANCELLED',
        'ReadyToShip': 'READY_TO_SHIP'
    }
    
    return status_mapping.get(trendyol_status, 'UNKNOWN')

# Test status mapping
test_statuses = ['Created', 'Shipped', 'Delivered', 'Cancelled', 'Unknown']
for status in test_statuses:
    print(f"Trendyol status: '{status}' → Internal status: '{map_order_status(status)}'")

Trendyol status: 'Created' → Internal status: 'NEW'
Trendyol status: 'Shipped' → Internal status: 'SHIPPED'
Trendyol status: 'Delivered' → Internal status: 'DELIVERED'
Trendyol status: 'Cancelled' → Internal status: 'CANCELLED'
Trendyol status: 'Unknown' → Internal status: 'UNKNOWN'


## Complete Order Normalization Function

Let's create a complete function to normalize Trendyol orders for our system.

In [6]:
def normalize_trendyol_orders(trendyol_orders, connection_id='TRENDYOL_DEFAULT'):
    """
    Normalizes Trendyol orders for our internal system
    
    Args:
        trendyol_orders (list): List of orders from Trendyol API
        connection_id (str): ID of the connection in our system
        
    Returns:
        list: Normalized orders
    """
    normalized_orders = []
    
    for order in trendyol_orders:
        # Extract phone number with our robust method
        phone_number = extract_phone_number(order)
        
        # Create normalized shipping details
        shipping_detail = {
            'recipientName': f"{order['shipmentAddress']['firstName']} {order['shipmentAddress']['lastName']}",
            'address': order['shipmentAddress']['address1'],
            'city': order['shipmentAddress']['city'],
            'state': order['shipmentAddress']['district'],
            'postalCode': order['shipmentAddress'].get('postalCode', ''),
            'country': 'Turkey',  # Trendyol operates in Turkey
            'phone': phone_number,
            'email': order.get('customerEmail', ''),
            'shippingMethod': order.get('cargoProviderName', '')
        }
        
        # Create normalized order
        normalized_order = {
            'platformOrderId': order['orderNumber'],
            'platformId': 'trendyol',
            'connectionId': connection_id,
            'orderDate': datetime.fromtimestamp(order['orderDate']/1000).isoformat(),
            'orderStatus': map_order_status(order['status']),
            'totalAmount': order.get('totalPrice', 0),
            'currency': 'TRY',  # Turkish Lira
            'shippingDetails': shipping_detail,
            'customerName': f"{order['customerFirstName']} {order['customerLastName']}",
            'customerEmail': order.get('customerEmail', ''),
            'customerPhone': phone_number,
            'notes': order.get('note', ''),
            'items': []
        }
        
        # Process order items
        for item in order['lines']:
            normalized_item = {
                'platformProductId': str(item.get('productCode', '')),
                'sku': item.get('merchantSku', ''),
                'barcode': item.get('barcode', ''),
                'title': item.get('productName', ''),
                'quantity': item.get('quantity', 0),
                'price': item.get('price', 0),
                'currency': 'TRY',
                'variantInfo': item.get('productSize', '')
            }
            normalized_order['items'].append(normalized_item)
        
        normalized_orders.append(normalized_order)
    
    return normalized_orders

# Test normalization on the first 2 orders
if len(sample_response['content']) >= 2:
    test_orders = sample_response['content'][:2]
    normalized = normalize_trendyol_orders(test_orders)
    print(f"Normalized {len(normalized)} orders")
    
    # Display the first normalized order
    import pprint
    pp = pprint.PrettyPrinter(indent=2, width=100)
    print("\nFirst normalized order:")
    pp.pprint(normalized[0])

## Analysis of Phone Number Coverage

Let's analyze how many orders have phone numbers after applying our extraction method.

In [7]:
if len(sample_response['content']) > 0:
    # Count original phone coverage
    original_phone_count = sum(1 for order in sample_response['content'] 
                               if order.get('shipmentAddress', {}).get('phone'))
    
    # Count extracted phone coverage
    extracted_phone_count = sum(1 for order in sample_response['content'] 
                               if extract_phone_number(order))
    
    total_orders = len(sample_response['content'])
    
    print(f"Original API phone coverage: {original_phone_count}/{total_orders} orders ({original_phone_count/total_orders*100:.1f}%)")
    print(f"After extraction phone coverage: {extracted_phone_count}/{total_orders} orders ({extracted_phone_count/total_orders*100:.1f}%)")
    print(f"Improvement: {extracted_phone_count - original_phone_count} additional phone numbers found")

Original API phone coverage: 0/1 orders (0.0%)
After extraction phone coverage: 1/1 orders (100.0%)
Improvement: 1 additional phone numbers found


## Conclusion and Implementation Notes

This notebook demonstrates how to handle missing phone numbers in Trendyol API responses. The key insights are:

1. The `phone` field in shipmentAddress is often null in Trendyol API responses
2. We can extract phone numbers from the fullAddress field using regex patterns
3. We can fallback to invoiceAddress phone if available
4. Our extraction method significantly improves phone number coverage

For production implementation:
- Add the `extractPhoneNumber` method to your TrendyolService class
- Use this method when normalizing orders
- Add unit tests to verify extraction effectiveness
- Consider adding more regex patterns based on your specific data

This approach ensures more complete data for customer contact information.